In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE117668"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE117668"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE117668.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE117668.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE117668.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE117668_family.soft.gz', 'GSE117668_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE117668/GSE117668_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE117668/GSE117668_series_matrix.txt.gz
Background Information:
!Series_title	"Expression data from in vitro healthy cells and malignant pleural mesothelioma cell lines infected by oncolytic attenuated measles virus or treated by exogenous type I interferon"
!Series_summary	"We used microarrays to analyse antiviral response by mesothelioma cells sensitive or resistant to the oncolytic activity of measles virus"
!Series_overall_design	"4 healthy cells and 12 malignant pleural mesothelioma cell lines were infected with measles virus (MV, MOI 1) or not infected (NI = basal expression) or were treated with type I interferon (IFN : IFN-alpha2 + IFN-Beta, 1000UI/mL) during 48 hours. Cells were lysed and RNA was extracted of each sample. This experiment was performed 3 times. RNA was quantif

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any, List
import re

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray expression data from cells
# This suggests gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Mesothelioma) can be inferred from diagnosis in row 1
trait_row = 1

# No age information is available in the sample characteristics
age_row = None

# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert diagnosis information to binary trait value (0 for healthy, 1 for mesothelioma)"""
    if not value or ':' not in value:
        return None
    
    # Extract the diagnosis value after the colon
    diagnosis = value.split(':', 1)[1].strip().lower()
    
    # Convert to binary
    if 'healthy' in diagnosis:
        return 0
    elif 'malignant pleural mesothelioma' in diagnosis:
        return 1
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Placeholder function for age conversion (not used in this dataset)"""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Placeholder function for gender conversion (not used in this dataset)"""
    return None

# Helper function to extract sample characteristics from GEO series matrix file
def get_feature_data(clinical_df, row_index, feature_name, converter):
    """Extract feature data from a row in clinical DataFrame and convert it."""
    feature_values = clinical_df.iloc[row_index, :].tolist()
    converted_values = [converter(val) for val in feature_values]
    return pd.DataFrame({feature_name: converted_values}, index=clinical_df.columns)

# 3. Save Metadata - Initial Filtering
# Determine if trait data is available based on whether trait_row is None
is_trait_available = trait_row is not None

# Save initial validation information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Process the GEO series matrix file to extract clinical data
    matrix_file_path = os.path.join(in_cohort_dir, "GSE117668_series_matrix.txt.gz")
    
    # Read lines from the file until we find the sample characteristics
    sample_char_lines = []
    with pd.io.common.get_handle(matrix_file_path, 'r', compression='gzip') as handle:
        file = handle.handle
        line = file.readline()
        while line:
            if line.startswith("!Sample_characteristics_ch"):
                sample_char_lines.append(line.strip())
            elif line.startswith("!Sample_geo_accession"):
                # Get the sample IDs
                sample_ids = line.strip().split('\t')[1:]
            elif line.startswith("!series_matrix_table_begin"):
                # We've reached the data section, stop parsing headers
                break
            line = file.readline()
    
    # Create a DataFrame for sample characteristics
    clinical_data = pd.DataFrame(index=sample_char_lines, columns=sample_ids)
    
    # Fill the DataFrame with values
    for i, line in enumerate(sample_char_lines):
        values = line.split('\t')[1:]  # Skip the first element (header)
        if len(values) == len(sample_ids):
            clinical_data.iloc[i] = values
    
    # Transpose to have samples as rows and characteristics as columns
    clinical_data = clinical_data.T
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    print("Preview of selected clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{'!Sample_characteristics_ch1\t"cell type: lung epithelial cells"\t"cell type: lung epithelial cells"\t"cell type: lung epithelial cells"\t"cell type: fibroblasts"\t"cell type: fibroblasts"\t"cell type: fibroblasts"\t"cell type: endothelial cells"\t"cell type: endothelial cells"\t"cell type: endothelial cells"\t"cell type: peritoneal mesothelial cells"\t"cell type: peritoneal mesothelial cells"\t"cell type: peritoneal mesothelial cells"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cell line"\t"cell type: MPM cel

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 29536
First 20 gene/probe identifiers:
Index(['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033413_at', '100033422_at', '100033423_at', '100033424_at',
       '100033425_at', '100033426_at', '100033427_at', '100033428_at',
       '100033430_at', '100033431_at', '100033432_at', '100033434_at',
       '100033435_at', '100033436_at', '100033437_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers pattern
# The identifiers follow a pattern like '100009613_at', '100009676_at', '10000_at'
# This format with "_at" suffix is typical of Affymetrix microarray probe IDs
# These are not standard human gene symbols and will need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at'], 'SPOT_ID': ['100009613', '100009676', '10000', '10001', '10002'], 'Description': ['ANO1 antisense RNA 2 (head to head)', 'ZBTB11 antisense RNA 1', 'AKT serine/threonine kinase 3', 'mediator complex subunit 6', 'nuclear receptor subfamily 2 group E member 3']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From the gene annotation preview, we see:
# - 'ID' contains identifiers like '100009613_at' which match the gene expression data identifiers
# - 'Description' contains what appears to be gene descriptions which likely include gene symbols

# First, let's better understand the structure of the Description column
print("Sample descriptions to analyze gene symbol format:")
print(gene_annotation['Description'].head(10))

# Map gene IDs to gene symbols
# Use ID as the probe ID column and Description as the gene symbol column
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Description')

# Print a preview of the mapping data
print("\nPreview of gene mapping data:")
print(preview_df(mapping_df))

# 2. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 3. Preview the resulting gene expression data
print("\nPreview of gene-level expression data:")
print(f"Shape: {gene_data.shape}")
print(f"Number of genes: {len(gene_data)}")
print(f"First 5 gene symbols: {list(gene_data.index[:5])}")
print(f"First 5 samples: {list(gene_data.columns[:5])}")

# 4. Save the gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Sample descriptions to analyze gene symbol format:
0              ANO1 antisense RNA 2 (head to head)
1                           ZBTB11 antisense RNA 1
2                    AKT serine/threonine kinase 3
3                       mediator complex subunit 6
4    nuclear receptor subfamily 2 group E member 3
5               small nucleolar RNA, C/D box 116-1
6              small nucleolar RNA, C/D box 116-10
7              small nucleolar RNA, C/D box 116-11
8              small nucleolar RNA, C/D box 116-12
9              small nucleolar RNA, C/D box 116-13
Name: Description, dtype: object

Preview of gene mapping data:
{'ID': ['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at'], 'Gene': ['ANO1 antisense RNA 2 (head to head)', 'ZBTB11 antisense RNA 1', 'AKT serine/threonine kinase 3', 'mediator complex subunit 6', 'nuclear receptor subfamily 2 group E member 3']}

Preview of gene-level expression data:
Shape: (3151, 48)
Number of genes: 3151
First 5 gene symbols: ['A-', 'A


Gene expression data saved to ../../output/preprocess/Mesothelioma/gene_data/GSE117668.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Create clinical data from the sample IDs in the gene expression data
# Since all samples are from tumor tissue, they all have mesothelioma (but this is not useful as a trait)
sample_ids = gene_data.columns.tolist()
print(f"Number of samples: {len(sample_ids)}")

# Extract gender information from the original matrix file
gender_data = {}
try:
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Read the matrix file to extract sample characteristics
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Display unique values in each row of clinical data
    characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample characteristics dictionary:")
    print(characteristics_dict)
    
    # Try to find gender information in the characteristics
    gender_row = None
    for idx, values in characteristics_dict.items():
        if any('sex:' in str(v).lower() for v in values):
            gender_row = idx
            break
    
    if gender_row is not None:
        # Extract gender data from the row
        for col in clinical_data.columns:
            if col != '!Sample_geo_accession':
                continue
                
            for idx, row in clinical_data.iterrows():
                if idx == gender_row:
                    for i, sample_id in enumerate(clinical_data.iloc[0].values):
                        if i > 0 and sample_id in sample_ids:  # Skip the first column (header)
                            gender_val = clinical_data.iloc[idx, i]
                            if isinstance(gender_val, str) and 'sex:' in gender_val.lower():
                                gender = 0 if 'f' in gender_val.lower() else 1 if 'm' in gender_val.lower() else None
                                gender_data[sample_id] = gender
except Exception as e:
    print(f"Error extracting gender data: {e}")

# Create a clinical dataframe
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df['Mesothelioma'] = 1  # All samples have mesothelioma

# Add gender if available
if gender_data:
    clinical_df['Gender'] = clinical_df.index.map(lambda x: gender_data.get(x))
    print(f"Added gender data for {sum(pd.notna(clinical_df['Gender']))} samples")

print("Clinical data preview:")
print(clinical_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data (transpose gene expression data to have samples as rows)
linked_data = pd.concat([clinical_df, gene_data_normalized.T], axis=1)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, 'Mesothelioma')
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait is biased (it will be since all samples are mesothelioma)
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Mesothelioma')

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,  # We do have trait data, it's just that all values are the same
    is_biased=is_trait_biased,  # This will be True since all samples have the same trait value
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from mesothelioma patients only (no controls), making trait biased."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to biased trait variable. Final linked data not saved.")

Shape of gene data after normalization: (2360, 48)
Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE117668.csv
Number of samples: 48


Sample characteristics dictionary:
{0: ['cell type: lung epithelial cells', 'cell type: fibroblasts', 'cell type: endothelial cells', 'cell type: peritoneal mesothelial cells', 'cell type: MPM cell line'], 1: ['diagnosis: healthy', 'diagnosis: malignant pleural mesothelioma\xa0']}
Clinical data preview:
            Mesothelioma
GSM3305861             1
GSM3305862             1
GSM3305863             1
GSM3305864             1
GSM3305865             1
Saved clinical data to ../../output/preprocess/Mesothelioma/clinical_data/GSE117668.csv
Shape of linked data: (48, 2361)


Shape of linked data after handling missing values: (48, 2361)
Quartiles for 'Mesothelioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

Dataset validation failed due to biased trait variable. Final linked data not saved.
